## Decision Trees (Árvores de Decisão)

**Importando o algoritmo**

    from sklearn.tree import DecisionTreeClassifier

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

As decision trees são modelos amplamente utilizados para tarefas de classificação e regressão.  
Essencialmente, eles aprendem através de uma hierarquia de perguntas “if-else”, levando a uma decisão.

Exemplo de decision tree para classificação de SPAM:
<img src="figuras/decision-trees.png">

Para construir uma árvore, o algoritmo pesquisa todos os testes possíveis e descobre aquele que é mais informativo sobre a variável target. Em seguida vai particionando os dados recursivamente de acordo com o resultado. Nesta ilustração, cada nó na árvore representa uma questão ou um nó terminal (também chamado de folha) que contém a resposta.

### Heurística
O Scikit-Learn usa o algoritmo CART, que produz apenas árvores binárias: os nós sempre têm dois filhos (ou seja, as perguntas têm apenas respostas sim / não). No entanto, outros algoritmos, como o ID3, podem produzir árvores de decisão com nós com mais de dois filhos.

**Como funciona:**
- Começa com todos os exemplos de treino;
- Escolhe o teste (atributo) que melhor divide os exemplos, ou seja agrupa exemplos da mesma classe ou exemplos semelhantes;
- Para o atributo escolhido, é criado um nó filho para cada valor possível do atributo;
- Transporta os exemplos para cada filho considerando o valor do filho;
- Repete o procedimento para cada filho não "puro".

**Greedy Selection**  
Como você pode ver, o algoritmo CART é um algoritmo guloso (greedy): procura avidamente por uma divisão ideal no nível superior e depois repete o processo em cada nível. Não verifica se a divisão levará ou não à mais baixa impureza possível vários níveis inferiores. Um algoritmo guloso geralmente produz uma solução razoavelmente boa, mas não é garantido que seja a solução ideal.

**Defição do nó raiz e e divisão do conjunto de dados**  
O algoritmo decide quais perguntas devem ser feitas em cada etapa através do ganho de informação (GI). Em cada nó, temos um certo número de instâncias (começando do conjunto de dados inteiro) e medimos sua **entropia**. O algoritimo realiza o cálculo da entropia antes e após o cálculo das partições e decide se mantém o nó se houver ganho de informação. O modelo vai sendo reajustado até que os nós que apresentam a maior redução na entropia sejam selecionados. Aos nós filhos que possuam entropia igual a zero, damos o nome de folha pura.

**Entropia**  
Na física entropia é a medida de grau de desordem de um sistema. Na teoria da informação, entropia é a medida da incerteza dos dados. Pode ser definida como o grau de impureza de um conjunto. A entropia é uma medida absoluta que fornece um número entre 0 e 1, independentemente do tamanho do conjunto.

**Portanto:**
* Grau máximo de entropia: Vários valores dentro da classe, sem predominância de nenhuma categoria.
* Grau mínimo de entropia: Todos os valores da classe são iguais;

Exemplo de um caso com 2 categorias:
<img src="figuras/entropia.png" width="700"/>

Fórmula da entropia:
<img src="figuras/entropia_formula.png" width="300"/>
**Onde:**

    n - Número de resultados  
    P(xi) - Probabilidade do resultado i  
*Nota: Valores comuns para b são 2, e (Número de Euler), e 10. Como o log de um número menor que um será negativo, a soma inteira é negativada para retornar um valor positivo.*  

Portanto, um conjunto de dados composto, por exemplo, de 25% de bolas azuis, 25% de bolas vermelhas, 25% de bolas verdes e 25% de bolas amarelas, teria entropia igual a 2.
<img src="figuras/entropia_formula2.png" width="700"/>

**Ganho de informação**  
É a redução esperada da entropia, ou seja, busca-se saber o nível de pureza dos dados, o que significa dados melhor organizados que permitem extrair as regras que eles representam. É calculado pela diferença entre a entropia do nó pai, H (T), e a média ponderada das entropias dos nós filhos.  

Digamos, por exemplo, que o nó-pai tenha entropia igual a 0,80 e que após a divisão, um nó filho fique com 80% das amostras e entropia de 0,60 e o segundo fique com 20% das amostras e entropia de 0,50. Calculando o ganho de informação teríamos:

    GI = 0,70 - (80% * 0,60) - (20% * 0,50)
    GI = 0,12

**Índice de Gini**  
O Índice de Gini é usado para medir a probabilidade de dois itens aleatórios pertencentes à mesma classe. A medida de Gini de um nó é a soma dos quadrados das proporções das classes.

**Nota:** Por padrão, o algoritmo usa os índice de Gini para determinar os nós de uma decision tree. A impureza de Gini é um pouco mais rápida para calcular, por isso é um bom padrão. Geralmente os resultados entre a entropia e Gini dão resultados semelhantes. No entanto, quando diferem, a impureza de Gini tende a isolar a classe mais frequente em seu próprio ramo da árvore, enquanto a entropia tende a produzir árvores ligeiramente mais balanceadas.

**Prouning**  
Normalmente, construir uma árvore como descrito acima e continuar até que todas as folhas sejam puras leva a modelos que são muito complexos e altamente adaptadas aos dados de treinamento (overfitting). A presença de folhas puras significa que uma árvore é 100% precisa no conjunto de treinamento.

Existem duas estratégias comuns para evitar o overfitting: parar a criação da árvore antes, também chamada de pré-poda (pre-pruning), ou construir a árvore, removendo ou colapsando nós que contêm pouca informação, também chamada de poda posterior (post-pruning) ou simplesmente de poda (pruning). Os critérios possíveis para a pré-poda incluem limitar a profundidade máxima da árvore, limitar o número máximo de folhas ou exigir um número mínimo de pontos em um nó para continuar dividindo-a.

As árvores de decisão no scikit-learn são implementadas nas classes DecisionTreeRegressor e DecisionTreeClassifier. O Scikit-learn apenas implementa a **pré-poda**, não a pós-poda.

Para limitar a quantidade de camadas de uma decision tree, configuramos o parâmetro ``max_depth``.

    tree = DecisionTreeClassifier(max_depth=4)
    tree.fit(X_train, y_train)

Outros parâmetros que podemos aplicar na pré-poda, para prevenir overfitting são:
* max_leaf_nodes - número máximo de nós folha;
* min_samples_leaf - o número mínimo de amostras que um nó folha deve ter;
* min_samples_split - o número mínimo de amostras que um nó deve ter antes de poder ser dividido;
* min_weight_fraction_leaf - o mesmo que min_samples_leaf mas expresso como uma fração do número total de instâncias ponderadas.

**Nota:** Aumentar o hiperparâmetro ``min_*`` ou reduzir o hiperparâmetro ``max_*`` regularizará o modelo.

### Visualizando as Decision Trees

Podemos exportar o resultado da árvore usando a função ``export_graphviz`` do ``sklearn.tree``.  
Essa função grava um arquivo no formato dot, que é um formato de arquivo de texto para armazenar gráficos.  

Para vizualizar o resultado, necessitamos dos pacotes **pydot** e **graphviz**. Para instalá-los utilize os comando abaixo:

``!pip install pydot`` -> Interface Python para GraphViz e a linguagem DOT  
``!pip install graphviz`` -> Biblioteca de vizualização

Em seguida necessitamos instalar o graphviz no Windows:  
https://graphviz.gitlab.io/download/


**Nota:** No Windows 10 foi necessário configurar as variáveis de sistema para que o Graphviz funcionasse. Para isso adicionamos no PATH de usuário e sistema o caminho onde o graphviz esta instalado: ``C:\Program Files (x86)\Graphviz2.38\bin``

### Importância dos Atributos

´Para analisar a árvore, em vez de olharmos seu diagrama, podemos também olhar a importância de cada atributo. Para isso utilizamos a função ``.feature_importances_``, que avalia a importância de cada atributo para a decisão de uma árvore. Essa função avalia a importância percentual de cada atributo, dando um peso entre 0 e 1, onde 0 significa “não utilizado” e 1 significa “prediz perfeitamente o alvo”.
Ao somarmos cada nota dada aos atributos obteremos a totalidade de 1, ou seja, 100%.

**Nota:** Se um atributo tiver uma feature_importance baixa, não significa que ele não seja informativo. Significa apenas que não foi escolhido pela árvore, provavelmente porque outro atributo já nos dê a mesma informação.

### Vantagens e desvantagens
**Vantagens**
* Modelo de fácil visualização e entendimento
* Processamento dos dados sem necessidade de normalização de escala

**Desvantagens**
* Tendência à overfit
* Performance de generalização pobre